# PREDICT USING THE MODEL
This is a notebook that loads the final pickled model and uses it with data from 2018 to predict which companies may delist and which companies are safe.

In [17]:
import numpy as np
import pandas as pd
import pickle

In [18]:
pipeline = pickle.load(open('./model_pickles/best_rf.pkl', 'rb'))

In [4]:
crystal_ball = pd.read_csv('../data/crystal_ball_web_ready.csv')
crystal_ball= crystal_ball.drop(columns='Unnamed: 0')
print(crystal_ball.shape)
crystal_ball.head()

(1613, 94)


,sa_finance1_cocode,sa_company_name,sa_total_income,sa_sales,sa_total_assets,sa_pat,sa_total_liabilities,sa_gross_working_capital_cosm,sa_current_assets_incl_st_invest_loans,sa_current_liabilities,...,sa_total_inc_net_of_pe_compensation_to_empl,sa_sales_avg_gfa_net_of_reval,sa_sales_net_repairs_avg_nfa_net_of_reval,equity_facevalue_on_stkdate,eps_on_stkdate,equity_bv_on_stkdate,high_price,stock_returns,market_cap,traded_qty
0,11,20 MICRONS LTD.,3922.0,3891.8,3808.6,158.8,3808.6,1573.9364,1679.5,1396.9,...,10.6027,1.5810,2.4380,5.0,5.5432,1382.7,68.00,1.1345,2244.2215,1151679.0
1,415,A B M KNOWLEDGEWARE LTD.,740.8,716.3,1976.5,144.5,1976.5,253.2485,1377.7,37.4,...,2.5029,1.4197,1.5215,5.0,7.2592,1493.2,142.00,1.1190,2711.2982,121094.0
2,912,A F ENTERPRISES LTD.,5.0,4.2,185.3,-24.0,185.3,143.7962,173.8,167.2,...,3.3333,1.2000,1.6154,10.0,-3.1250,17.3,5.96,1.0442,23.8400,7040.0
3,1120,A I A ENGINEERING LTD.,23947.3,21658.2,43578.4,4699.1,43578.4,10158.3090,25120.5,3406.7,...,29.0374,2.6616,3.3295,2.0,52.0418,28657.6,1887.00,1.0535,172120.5272,3135822.0
4,1144,A INFRASTRUCTURE LTD.,2163.8,2138.8,2100.4,8.1,2100.4,1117.7045,1332.7,1021.9,...,11.3207,4.6486,5.0942,10.0,1.8103,485.9,149.35,1.0500,1592.2838,4080.0


In [6]:
def make_prediction(company_name):   
    pull_out = crystal_ball.loc[crystal_ball['sa_company_name']==company_name,'sa_total_income':]
    X = np.array(pull_out)
    prob_delisted = pipeline.predict_proba(X)[0, 1]
    result = [company_name,int(prob_delisted > 0.15),prob_delisted]
    return result




In [20]:
if __name__ == '__main__':
    company_list = list(crystal_ball['sa_company_name'])
    future =[]
    for c in company_list:           
        future.append(make_prediction(c))        

    delist= []
    for company in future:
        if (company[1]==1):
            delist.append(company)
    
    df_unsafe = pd.DataFrame(delist, columns=["company_name", "prediction", "probability"])

In [21]:
# companies not to invest in
df_unsafe.head(10)

,company_name,prediction,probability
0,A F ENTERPRISES LTD.,1,0.375683
1,A V I POLYMERS LTD.,1,0.745590
2,ACE SOFTWARE EXPORTS LTD.,1,0.179590
3,ADANI POWER LTD.,1,0.257816
4,ADHARSHILA CAPITAL SERVICES LTD.,1,0.814206
5,ARIS INTERNATIONAL LTD.,1,0.256817
6,ADVENT COMPUTER SERVICES LTD.,1,0.194174
7,HUBTOWN LTD.,1,0.413894
8,AMARNATH SECURITIES LTD.,1,0.243554
9,ANAND PROJECTS LTD.,1,0.159191


In [23]:
df_unsafe.shape

(271, 3)